In [ ]:
import streamlit as st
import pandas as pd
import requests
import ast

In [ ]:
nested_egg = pd.read_json('https://api.flipsidecrypto.com/api/v2/queries/290a91fb-8186-486d-992e-985839352809/data/latest')
loot = pd.read_json('https://api.flipsidecrypto.com/api/v2/queries/2208cc91-e624-4b40-b666-f65c8b6405f8/data/latest')
eggs = pd.read_json('https://api.flipsidecrypto.com/api/v2/queries/ee319c5f-4f03-41b5-b607-99f306fe1e2f/data/latest')
meteor_dust = pd.read_json('https://api.flipsidecrypto.com/api/v2/queries/8eebec3c-db0e-4051-a5c2-50a5deb1070b/data/latest')
meteors = pd.read_json('https://api.flipsidecrypto.com/api/v2/queries/b9778a1d-9b63-4ba5-a48f-7b28569b5603/data/latest')
#IT IS ABSOLUTELY CRUCIAL THAT YOU DO NOT CHANGE THE ORIGINAL NAMES OF THE DATAFRAME COLUMNS ON VELOCITY

#MUST STORE IN A DICTIONARY 
current_nft_dict = {'nested_egg':nested_egg, 'loot':loot, 'eggs':eggs, 'meteor_dust':meteor_dust, 'meteors':meteors}



In [ ]:
nft_luna_price_df = []
for nft in current_nft_dict:
    #renames columns from Flipside dataframes according to NFT
    nft_luna_price_df.append(current_nft_dict[nft][['BLOCK_TIMESTAMP', 'NFT_LUNA_PRICE']].rename({'NFT_LUNA_PRICE':f'{nft}'}, axis = 1))
nft_luna_price_df = pd.concat(nft_luna_price_df)
nft_luna_price_df['BLOCK_TIMESTAMP'] = pd.to_datetime(nft_luna_price_df['BLOCK_TIMESTAMP'])
nft_luna_price_df.set_index('BLOCK_TIMESTAMP', inplace = True)
nft_luna_price_df.index = nft_luna_price_df.index.round('D')

# START HERE

In [ ]:
nft_luna_price_df = []
for nft in current_nft_dict:
    #renames columns from Flipside dataframes according to NFT
    nft_luna_price_df.append(current_nft_dict[nft][['BLOCK_TIMESTAMP', 'NFT_TYPE', 'NFT_LUNA_PRICE', 'NFT_UST_PRICE_AT_PURCHASE']])

nft_luna_price_df = pd.concat(nft_luna_price_df)
nft_luna_price_df['BLOCK_TIMESTAMP'] = pd.to_datetime(nft_luna_price_df['BLOCK_TIMESTAMP'])
nft_luna_price_df.set_index('BLOCK_TIMESTAMP', inplace = True)
nft_luna_price_df.index = nft_luna_price_df.index.round('D')
nft_luna_price_df


In [ ]:
total_df = nft_luna_price_df.groupby(['BLOCK_TIMESTAMP', 'NFT_TYPE']).sum().rename(columns={'NFT_LUNA_PRICE':'TOTAL_LUNA', 'NFT_UST_PRICE_AT_PURCHASE':'TOTAL_UST'})
average_df = nft_luna_price_df.groupby(['BLOCK_TIMESTAMP', 'NFT_TYPE']).mean().rename(columns={'NFT_LUNA_PRICE':'AVERAGE_LUNA', 'NFT_UST_PRICE_AT_PURCHASE':'AVERAGE_UST'})
tx_count_df = nft_luna_price_df.groupby(['BLOCK_TIMESTAMP', 'NFT_TYPE']).count().rename(columns={'NFT_LUNA_PRICE':'TRANSACTION_COUNT'})
tx_count_df = tx_count_df['TRANSACTION_COUNT']
min_df = nft_luna_price_df.groupby(['BLOCK_TIMESTAMP', 'NFT_TYPE']).min().rename(columns={'NFT_LUNA_PRICE':'MIN_LUNA', 'NFT_UST_PRICE_AT_PURCHASE':'MIN_UST'})
max_df = nft_luna_price_df.groupby(['BLOCK_TIMESTAMP', 'NFT_TYPE']).max().rename(columns={'NFT_LUNA_PRICE':'MAX_LUNA', 'NFT_UST_PRICE_AT_PURCHASE':'MAX_UST'})
median_df = nft_luna_price_df.groupby(['BLOCK_TIMESTAMP', 'NFT_TYPE']).median().rename(columns={'NFT_LUNA_PRICE':'MEDIAN_LUNA', 'NFT_UST_PRICE_AT_PURCHASE':'MEDIAN_UST'})
cum_sum = nft_luna_price_df.groupby(['BLOCK_TIMESTAMP', 'NFT_TYPE']).sum().rename(columns={'NFT_LUNA_PRICE':'TOTAL_LUNA_CUMULATIVE', 'NFT_UST_PRICE_AT_PURCHASE':'TOTAL_UST_CUMULATIVE'}).cumsum(axis=0)


master_df = pd.concat([total_df, tx_count_df, average_df, min_df, max_df, median_df, cum_sum], axis = 1)
master_df

# RARITY TABLE - Meteors, Eggs, Dust


In [ ]:
# nft_luna_price_df = []
# for nft in current_nft_dict:
#     #renames columns from Flipside dataframes according to NFT
#     nft_luna_price_df.append(current_nft_dict[nft][['BLOCK_TIMESTAMP', 'NFT_LUNA_PRICE']].rename({'NFT_LUNA_PRICE':f'{nft}'}, axis = 1))
# nft_luna_price_df = pd.concat(nft_luna_price_df)
# nft_luna_price_df['BLOCK_TIMESTAMP'] = pd.to_datetime(nft_luna_price_df['BLOCK_TIMESTAMP'])
# nft_luna_price_df.set_index('BLOCK_TIMESTAMP', inplace = True)
# nft_luna_price_df.index = nft_luna_price_df.index.round('D')

#meteors
meteor_rarity = pd.read_json('https://api.flipsidecrypto.com/api/v2/queries/5d7b0f96-a537-46e5-9b3c-d5145072279f/data/latest')
meteor_rarity_merge = pd.merge(meteors, meteor_rarity[['TOKEN_ID', 'RARITY']], on='TOKEN_ID')
egg_rarity = pd.read_json('https://api.flipsidecrypto.com/api/v2/queries/284600cc-7b4c-4d47-ab50-dbdd528daf93/data/latest')
egg_rarity_merge = pd.merge(eggs, egg_rarity[['TOKEN_ID', 'RARITY']], on='TOKEN_ID')
dust_rarity = pd.read_json('https://api.flipsidecrypto.com/api/v2/queries/f4b78c34-07ec-4f5e-8099-7e6db2db24a9/data/latest')
dust_rarity_merge = pd.merge(meteor_dust, dust_rarity[['TOKEN_ID', 'RARITY']], on='TOKEN_ID')




# RARITY - Nested Eggs, Loot

In [ ]:
#Alternative using swagger; this method takes an exceptionally large amount of time
def swagger_app(num):
        rarity = requests.get(f"https://fcd.terra.dev/wasm/contracts/terra1vhuyuwwr4rkdpez5f5lmuqavut28h5dt29rpn6/store?query_msg=%7B%22nft_info%22:%7B%22token_id%22:%22{num}%22%7D%7D").json()
        return rarity.get('result').get('extension').get('attributes')[1].get('value')
        #print(num)

nested_egg['RARITY'] = nested_egg['TOKEN_ID'].apply(lambda x: swagger_app(x))


In [ ]:
#for nested eggs
#This should be the faster version pulling directly from the server set up to scrape RE
nested_rarity = pd.read_csv('http://165.22.125.123/nested_egg_nfts.csv ')
nested_rarity_merge = pd.merge(nested_egg, nested_rarity[['token_id', 'traits']], left_on='TOKEN_ID', right_on='token_id', how='inner')
nested_rarity_merge['RARITY'] = nested_rarity_merge['traits'].apply(lambda x: ast.literal_eval(x).get('Rarity'))
nested_rarity_merge.pop('traits')
nested_rarity_merge

In [ ]:
loot_rarity = pd.read_csv('http://165.22.125.123/loot_nfts.csv')
loot_rarity_merge = pd.merge(loot, loot_rarity[['token_id', 'traits']], left_on='TOKEN_ID', right_on='token_id', how='inner')
loot_rarity_merge['RARITY'] = loot_rarity_merge['traits'].apply(lambda x: ast.literal_eval(x).get('Rarity'))
loot_rarity_merge.pop('traits')
loot_rarity_merge

In [ ]:
nft_luna_price_df = []
for nft in current_nft_dict:
    #renames columns from Flipside dataframes according to NFT
    nft_luna_price_df.append(current_nft_dict[nft][['BLOCK_TIMESTAMP', 'NFT_TYPE', 'NFT_LUNA_PRICE']].rename({'NFT_LUNA_PRICE':f'{nft}_LUNA_PRICE'}, axis = 1))
nft_luna_price_df = pd.concat(nft_luna_price_df, axis= 1)
nft_luna_price_df['BLOCK_TIMESTAMP'] = pd.to_datetime(nft_luna_price_df['BLOCK_TIMESTAMP'])
nft_luna_price_df.set_index('BLOCK_TIMESTAMP', inplace = True)
nft_luna_price_df.index = nft_luna_price_df.index.round('D')
nft_luna_price_df.groupby('BLOCK_TIMESTAMP').sum()

In [ ]:
nft_ust_price_df = []
for nft in current_nft_dict:
    #renames columns from Flipside dataframes according to NFT
    nft_ust_price_df.append(current_nft_dict[nft][['BLOCK_TIMESTAMP', 'NFT_UST_PRICE_AT_PURCHASE']].rename({'NFT_UST_PRICE_AT_PURCHASE':f'{nft}_UST_PRICE'}, axis = 1))
nft_ust_price_df = pd.concat(nft_ust_price_df)
nft_ust_price_df['BLOCK_TIMESTAMP'] = pd.to_datetime(nft_ust_price_df['BLOCK_TIMESTAMP'])
nft_ust_price_df.set_index('BLOCK_TIMESTAMP', inplace = True)
nft_ust_price_df.index = nft_ust_price_df.index.round('D')
nft_ust_price_df.groupby('BLOCK_TIMESTAMP').sum()

In [ ]:
#pd.DataFrame(nft_ust_price_df.sum()).reset_index().rename(columns={0:"UST Exchanged Sum"})
nft_ust_price_df.sum()

In [ ]:
tx_count = pd.DataFrame(nft_ust_price_df.apply(lambda x: (x > 0).sum(), axis =  1).groupby('BLOCK_TIMESTAMP').sum()).reset_index()
pd.DataFrame(tx_count.rename(columns={0:"Transaction Count"})['Transaction Count'])

In [ ]:
nft_luna_price_df.index = nft_luna_price_df.index.round('H')
nft_luna_price_df.groupby('BLOCK_TIMESTAMP').sum()